In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob
import cv2

print("open cv2 version: %s" % cv2.__version__)

open cv2 version: 3.3.1


In [16]:
def load_imgs(vidPath, firstFrame, lastFrame):
    '''
    Reads frames from video and stores as a list
    
    Parameters: 
    vidPath (string): Path to video
    
    Returns: 
    list of images
    '''
    
    cap = cv2.VideoCapture(vidPath)
    length = np.arange(start, stop)
    print(len(length))
    imgs = []
    for ff in length:
        cap.set(1,ff)
        ret, frame = cap.read()
        if np.mod(ff, 100) == 0:
            print(ff) # prints progress in 50 frames

            # convert to grey
            #img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        imgs.append(frame)
#         print("length of imgs: %s" % len(imgs))
            
    cap.release()
    return imgs

In [17]:
# adjust gamma, if your vid is too dark
def adjust_gamma(image, gamma=1.0):
    # build a lookup table mapping the pixel values [0, 255] to
    # their adjusted gamma values
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255
        for i in np.arange(0, 256)]).astype("uint8")
 
    # apply gamma correction using the lookup table
    return cv2.LUT(image, table)

In [18]:
# list of mothIDs that require video making

guide = pd.read_csv("../dataFolders/PaperPipelineOutput/FilteredTracks_v2/FirstVisit/AllVideoNames.csv")
print(guide.head())
filesToRead = guide[guide.AutomatatedTracking != "TRUE"]

pathForVideo = r"G:\My Drive\Tom-Tanvi\Moth Learning Project\AllVideosForAnalysis\CompliedDataForAnalysis"

ourVideoPath = glob.glob(pathForVideo + "\**\\*.mp4", recursive = True)

   Unnamed: 0     names AutomatatedTracking
0           0  c-10_m10                TRUE
1           1  c-10_m11                TRUE
2           2  c-10_m12         VIDEO check
3           3  c-10_m13         VIDEO check
4           4  c-10_m15         VIDEO check


In [19]:
# get the tracks

direc = r"../dataFolders/PaperPipelineOutput/FilteredTracks_v2/ManualCleanup/FinalTracks/FirstVisit/"
AllTracks = glob.glob(direc + "*.csv")

In [20]:
# use step 5 in moth learning to read out all the start stop frame numbers

path_for_visit_frames = r"../../MothLearning/dataFolders/Output/Step5_FilesWith_TrueTrialAnd_ProboscisDetect_v2/"

In [21]:
# open the video and create an image stack of relevant frames
for mothID in filesToRead.names.values[0:1]:

    mothID = 'c-3_m10'
# mothID = filesToRead.names.values[0]
# mothID = 'c-1_m13'
    SpecificVideoPath = [f for f in ourVideoPath if mothID in f][0]

    if not os.path.isfile(SpecificVideoPath): print("video doesn not exist")


    path_frame_Reference = glob.glob(path_for_visit_frames + mothID + '*RawDataForExplorationTime.csv')
    Visit_info = pd.read_csv(path_frame_Reference[0])
    Visit_info = Visit_info[['MothIN', 'MothOut','ProboscisDetect']]

    visit_num = 0
    start = Visit_info.loc[visit_num, 'MothIN']
    stop = Visit_info.loc[visit_num, 'ProboscisDetect']
    if np.isnan(stop):
        stop = Visit_info.loc[visit_num, 'MothOut']


    # get the list of images

    # get vid info
    cap = cv2.VideoCapture(SpecificVideoPath)

    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps    = cap.get(cv2.CAP_PROP_FPS)

    print(length, width, height, fps)
    cap.release()

    # load images
    imList = load_imgs(SpecificVideoPath, start, stop)

    print("length of section you want to read: %s" % int(stop - start))
    print("length of the image list: %s" % len(imList))
    
    # get the track for that mothID
    specificTrack = [f for f in AllTracks if mothID in f][0]
    track = pd.read_csv(specificTrack)
    x = track.x.values
    y = track.y.values

    print('length of track is: ' + str(len(x)))
    
    # trancate the image list to track length
    imList = imList[:len(x)]
    
    
    # # make directory to store images
    tempImgDirectory = os.path.join(os.path.dirname(r"../dataFolders/PaperPipelineOutput/Figures/v2/Videos/"), "tmpImgs")
    if not os.path.exists(tempImgDirectory):
        os.mkdir(tempImgDirectory)

    # add circles up to XX points in the past

    lagPoints = 50 # number of points to plot on each frame
    alphaVals = np.flip(np.linspace(0, 0.2, lagPoints), axis = 0)
    vidLen = len(imList)
    total_zoom = 500

    outputImgs = []
    for frameNum in np.arange(0, vidLen):

        # adjust gamma
        image = adjust_gamma(imList[frameNum], 1.5)

        # convert to rgb, so I can add colored points
        #image = cv2.cvtColor(im1,cv2.COLOR_GRAY2RGB)

        overlay = image.copy()
        output = image.copy()
        for jj in range(lagPoints):
            if np.isnan(x[np.max([frameNum-jj, 0])]) or np.isnan(y[np.max([frameNum-jj, 0])]) :
                continue                                          
                                                
            # add circle, centered at closest integer value
            cv2.circle(overlay, (int(x[np.max([frameNum-jj, 0])]), 
                                 int(y[np.max([frameNum-jj, 0])])), 
                       3, (219, 189, 166), -1)
            cv2.addWeighted(overlay, alphaVals[jj], output, 1 - alphaVals[jj],0, output)

        f = plt.figure(figsize = (8, 3))
        ax0 = f.add_subplot(121)
        ax1 = f.add_subplot(122)

        ax0.imshow(output)
        ax0.set_xticks([])
        ax0.set_yticks([])

        if len(track) < total_zoom:
            (zoom_in_str, zoom_in_stop) = (0,vidLen)
        else:
            if frameNum > 20:
                (zoom_in_str, zoom_in_stop) = (frameNum - 20, frameNum + total_zoom - 20)
                if frameNum + total_zoom > vidLen:
                    (zoom_in_str, zoom_in_stop) = (vidLen-total_zoom,vidLen)
            else:
                (zoom_in_str, zoom_in_stop) = (frameNum,frameNum+total_zoom)

        x_range = np.arange(zoom_in_str, zoom_in_stop)*1/100
        ax1.plot(x_range, track.r[zoom_in_str:zoom_in_stop], 'k')
        ax1.plot(frameNum*1/100, track.r[frameNum], 'ob')
        ax1.set_ylim(0,5)
        ax1.set_xlabel('Time (sec)')
        ax1.set_ylabel('Radial Distance')

        plt.tight_layout(pad=0.1, h_pad=None, w_pad=0.5, rect=None)

    # use GridSpec for controlling the spacing of the subplots

        # display image -- it may be a bit faster if you don't display the images
    #     cv2.imshow(output) 

    #     save img
        plt.savefig(os.path.join(tempImgDirectory, str(frameNum).zfill(4) + ".png"))
        plt.close()

        if np.mod(frameNum, 100) == 0:
            print(frameNum, " of ", vidLen)
            
    # convert images to video with ffmpeg
    os.chdir(tempImgDirectory)
    print(tempImgDirectory)

    # use ffmpeg to convert directory to video
    # -r is output frame rate
    os.system('ffmpeg -start_number 0 -r 20 -i %04d.png -vf "scale=trunc(iw/2)*2:trunc(ih/2)*2" -c:v libx264 -pix_fmt yuv420p -y outputVid.mp4')
    nameofVideo = r'./' + mothID    + '.mp4'
    os.rename(r'./outputVid.mp4', nameofVideo)
    os.chdir('../../../../../../PaperPinelineCodes/')

    # delete images from directory
    delFiles = [f for f in os.listdir(tempImgDirectory) if f.endswith("png")]

    dlfs = [os.remove(os.path.join(tempImgDirectory, delFiles[ii])) for ii in range(len(delFiles))]

183794 648 488 30.0
1088
20300.0
20400.0
20500.0
20600.0
20700.0
20800.0
20900.0
21000.0
21100.0
21200.0
21300.0
length of section you want to read: 1088
length of the image list: 1088
length of track is: 879
0  of  879
100  of  879
200  of  879
300  of  879
400  of  879
500  of  879
600  of  879
700  of  879
800  of  879
../dataFolders/PaperPipelineOutput/Figures/v2/Videos\tmpImgs
